<a href="https://colab.research.google.com/github/JacopoMangiavacchi/FastLabeling/blob/master/mnist_odd_even_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install 'fastai2>=0.0.11'

In [44]:
from fastai2.vision.all import *

matplotlib.rc('image', cmap='Greys')

In [45]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
path.ls()

(#4) [Path('oe_training'),Path('oe_testing'),Path('training'),Path('testing')]

In [46]:
(path/'training').ls()

(#10) [Path('training/0'),Path('training/1'),Path('training/7'),Path('training/8'),Path('training/6'),Path('training/3'),Path('training/2'),Path('training/9'),Path('training/4'),Path('training/5')]

In [57]:
import shutil
from shutil import copy2

def copy_files(path, source_folder, destination_folder, sub_folders_list = [], random_select = False, max_files_to_copy = 0):
  def copy_folder_path(source_path, destination_path):
    source_files = source_path.ls()
    list_files = list(range(len(source_files)))
    if random_select:
      random.shuffle(list_files)
    count = 0
    for i in list_files:
      copy2(source_files[i], destination_path)
      count += 1
      if max_files_to_copy > 0 and count >= max_files_to_copy:
        break
  
  (path/destination_folder).mkdir(exist_ok=True)
  if len(sub_folders_list) > 0:
    for i in sub_folders_list:
      folder = source_folder + '/' + str(i)
      copy_folder_path(path/folder, path/destination_folder)
  else:
    copy_folder_path(path/source_folder, path/destination_folder)


In [58]:
shutil.rmtree(path/'oe_training')
shutil.rmtree(path/'oe_testing')

(path/'oe_training').mkdir(exist_ok=True)
(path/'oe_testing').mkdir(exist_ok=True)
path.ls()

(#6) [Path('oe_training'),Path('30_training'),Path('oe_testing'),Path('training'),Path('testing'),Path('30_testing')]

In [59]:
copy_files(path, 'training', 'oe_training/odd', [0, 2, 4, 6, 8])
copy_files(path, 'training', 'oe_training/even', [1, 3, 5, 7, 9])
copy_files(path, 'testing', 'oe_testing/odd', [0, 2, 4, 6, 8])
copy_files(path, 'testing', 'oe_testing/even', [1, 3, 5, 7, 9])


In [60]:
(path/'oe_training/odd').ls(), (path/'oe_training/even').ls(), (path/'oe_testing/odd').ls(), (path/'oe_testing/even').ls()

((#29492) [Path('oe_training/odd/55152.png'),Path('oe_training/odd/59372.png'),Path('oe_training/odd/17528.png'),Path('oe_training/odd/51986.png'),Path('oe_training/odd/47486.png'),Path('oe_training/odd/40749.png'),Path('oe_training/odd/49398.png'),Path('oe_training/odd/37646.png'),Path('oe_training/odd/37184.png'),Path('oe_training/odd/51565.png')...],
 (#30508) [Path('oe_training/even/59242.png'),Path('oe_training/even/26571.png'),Path('oe_training/even/47314.png'),Path('oe_training/even/3751.png'),Path('oe_training/even/44687.png'),Path('oe_training/even/5668.png'),Path('oe_training/even/31134.png'),Path('oe_training/even/780.png'),Path('oe_training/even/8330.png'),Path('oe_training/even/44534.png')...],
 (#4926) [Path('oe_testing/odd/8330.png'),Path('oe_testing/odd/6596.png'),Path('oe_testing/odd/6547.png'),Path('oe_testing/odd/542.png'),Path('oe_testing/odd/5393.png'),Path('oe_testing/odd/285.png'),Path('oe_testing/odd/8310.png'),Path('oe_testing/odd/156.png'),Path('oe_testing/odd

In [66]:
shutil.rmtree(path/'30_training')
shutil.rmtree(path/'30_testing')

(path/'30_training').mkdir(exist_ok=True)
(path/'30_testing').mkdir(exist_ok=True)
path.ls()

(#6) [Path('oe_training'),Path('30_training'),Path('oe_testing'),Path('training'),Path('testing'),Path('30_testing')]

In [70]:
copy_files(path, 'training/0', '30_training/0', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/1', '30_training/1', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/2', '30_training/2', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/3', '30_training/3', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/4', '30_training/4', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/5', '30_training/5', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/6', '30_training/6', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/7', '30_training/7', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/8', '30_training/8', random_select=True, max_files_to_copy=30)
copy_files(path, 'training/9', '30_training/9', random_select=True, max_files_to_copy=30)


In [ ]:
dls = ImageDataLoaders.from_folder(path, train='oe_training', valid='oe_testing')

dls.train_ds, dls.valid_ds

((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(3)


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.299438,0.218618,0.913800,01:17


epoch,train_loss,valid_loss,accuracy,time
0,0.050444,0.028868,0.989200,01:24
1,0.025768,0.018245,0.994200,01:24
2,0.009342,0.014318,0.995600,01:24


In [ ]:
dls2 = ImageDataLoaders.from_folder(path, train='training', valid='testing')

dls2.train_ds, dls.valid_ds

((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size

In [ ]:
learn2 = cnn_learner(dls2, resnet34, metrics=accuracy, custom_head=learn.model)
learn2.fine_tune(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.608689,0.409924,0.869700,01:16


epoch,train_loss,valid_loss,accuracy,time
0,0.125913,0.065786,0.978100,01:23
1,0.065100,0.040316,0.988400,01:23
2,0.033875,0.031843,0.989400,01:22
3,0.017199,0.022645,0.993100,01:22
4,0.011985,0.019886,0.994200,01:23


In [ ]:
test_path = (path/'testing'/'5').ls()[10]
test_image = Image.open(test_path)
test_image

In [ ]:
learn2.predict(test_path)

('5',
 tensor(5),
 tensor([2.1998e-07, 1.3172e-07, 3.1347e-06, 8.4968e-05, 2.5196e-07, 9.9990e-01,
         1.8432e-06, 1.2153e-06, 5.9199e-06, 1.5777e-07]))